# Import Libraries
In this section we import the libraries that will be required to process the data.

The first library is Pandas. Pandas is an open source, BSD-licensed library, providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

In [9]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopy
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

!pip install folium
import folium

print('Libraries imported.')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 8.0 MB/s  eta 0:00:01
Libraries imported.


# Download and Explore Dataset
Download and Explore Dataset Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a wget command and access the data. So let's go ahead and do that

In [10]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


# Tranform the data into a pandas dataframe

In [11]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


# Use geopy library to get the latitude and longitude values of New York City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [13]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [14]:

manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


# Create a map of New York with neighborhoods superimposed on top.

In [15]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# Foursquare venues

In [16]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [17]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [18]:
#https://developer.foursquare.com/docs/resources/categories
#Sushi = 4bf58dd8d48988d1d2941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_sushi = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1d2941735')
newyork_venues_sushi.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Planet Tokyo,40.886233,-73.909479,Sushi Restaurant
1,Chinatown,40.715618,-73.994279,Sushi Hatsune,40.715994,-73.992859,Sushi Restaurant
2,Chinatown,40.715618,-73.994279,Mikaku Sushi,40.721419,-73.996731,Sushi Restaurant
3,Chinatown,40.715618,-73.994279,Nakaji,40.715912,-73.996597,Sushi Restaurant
4,Chinatown,40.715618,-73.994279,Bondi Bar,40.721247,-73.996264,Sushi Restaurant


In [19]:
newyork_venues_sushi.shape

(1100, 7)

In [20]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [21]:
map_newyork_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_sushi, 'red', map_newyork_sushi)

map_newyork_sushi

In [22]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [23]:
manhattan_grouped = newyork_venues_sushi.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_sushi['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,22,22,22,22,22,22
Carnegie Hill,24,24,24,24,24,24
Central Harlem,3,3,3,3,3,3
Chelsea,41,41,41,41,41,41
Chinatown,25,25,25,25,25,25
Civic Center,32,32,32,32,32,32
Clinton,36,36,36,36,36,36
East Harlem,3,3,3,3,3,3
East Village,50,50,50,50,50,50


# Analyze Each Neighborhood

In [24]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_sushi[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_sushi['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Karaoke Bar,Noodle House,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [25]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Karaoke Bar,Noodle House,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.045455,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.863636,0.000000
1,Carnegie Hill,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.125000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.750000,0.041667
2,Central Harlem,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000
3,Chelsea,0.048780,0.00,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0.000000,0.000000,0.097561,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.804878,0.024390
4,Chinatown,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.120000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.880000,0.000000
5,Civic Center,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.031250,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.906250,0.000000
6,Clinton,0.027778,0.00,0.000000,0.027778,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.027778,0.000000,0.000000,0.805556,0.000000
7,East Harlem,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000
8,East Village,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.900000,0.020000
9,Financial District,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.181818,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.818182,0.000000


In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,Indian Chinese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market
1,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Indian Chinese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Sandwich Place,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar
2,Central Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
3,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Hawaiian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar
4,Chinatown,Sushi Restaurant,Japanese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant


# Cluster Neighborhoods

In [28]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 1, 0, 1, 4, 4, 2, 0, 4, 2], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,4,Sushi Restaurant,Japanese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
2,Washington Heights,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
3,Inwood,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
4,Hamilton Heights,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
5,Manhattanville,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
6,Central Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
7,East Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
18,Greenwich Village,Sushi Restaurant,Japanese Restaurant,Sake Bar,Indian Chinese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market
21,Tribeca,Sushi Restaurant,Noodle House,Vegetarian / Vegan Restaurant,Indian Chinese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market


In [32]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Hawaiian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar
9,Yorkville,Sushi Restaurant,Japanese Restaurant,Fish Market,Indian Chinese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Grocery Store,Bakery,Bubble Tea Shop,Chinese Restaurant
10,Lenox Hill,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Hawaiian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar
15,Midtown,Sushi Restaurant,Asian Restaurant,Seafood Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Restaurant,Ramen Restaurant,Sandwich Place,Fish Market
16,Murray Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Restaurant,Ramen Restaurant,Karaoke Bar,Grocery Store,Bubble Tea Shop
17,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Hawaiian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar
30,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Indian Chinese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Sandwich Place,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar
33,Midtown South,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Restaurant,Ramen Restaurant,Grocery Store,Bubble Tea Shop,Chinese Restaurant
34,Sutton Place,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Deli / Bodega,Steakhouse,Seafood Restaurant,Smoothie Shop,Grocery Store,Bakery,Bubble Tea Shop


In [33]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Sushi Restaurant,Japanese Restaurant,Smoothie Shop,Chinese Restaurant,Grocery Store,Indian Chinese Restaurant,Bakery,Bubble Tea Shop,Cocktail Bar,Deli / Bodega
14,Clinton,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Cocktail Bar,Asian Restaurant,Seafood Restaurant,Sandwich Place,Grocery Store,Bakery,Bubble Tea Shop
20,Lower East Side,Sushi Restaurant,Japanese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
25,Manhattan Valley,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store
26,Morningside Heights,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store
27,Gramercy,Sushi Restaurant,Japanese Restaurant,Bubble Tea Shop,Deli / Bodega,Vegetarian / Vegan Restaurant,Sandwich Place,Grocery Store,Bakery,Steakhouse,Chinese Restaurant
29,Financial District,Sushi Restaurant,Japanese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
38,Flatiron,Sushi Restaurant,Japanese Restaurant,Deli / Bodega,Vegetarian / Vegan Restaurant,Asian Restaurant,Sandwich Place,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar


In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,Sushi Restaurant,Japanese Restaurant,Grocery Store,Asian Restaurant,Seafood Restaurant,Sandwich Place,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar


In [35]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Sushi Restaurant,Japanese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
11,Roosevelt Island,Sushi Restaurant,Asian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
19,East Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Smoothie Shop,Hawaiian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega
22,Little Italy,Sushi Restaurant,Japanese Restaurant,Noodle House,Indian Chinese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market
23,Soho,Sushi Restaurant,Japanese Restaurant,Noodle House,Indian Chinese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market
24,West Village,Sushi Restaurant,Japanese Restaurant,Sake Bar,Fish Market,Vegetarian / Vegan Restaurant,Asian Restaurant,Grocery Store,Bakery,Bubble Tea Shop,Chinese Restaurant
28,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,Indian Chinese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market
31,Noho,Sushi Restaurant,Japanese Restaurant,Fish Market,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
32,Civic Center,Sushi Restaurant,Japanese Restaurant,Noodle House,Indian Chinese Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market
35,Turtle Bay,Sushi Restaurant,Japanese Restaurant,Seafood Restaurant,Karaoke Bar,Asian Restaurant,Steakhouse,Sandwich Place,Grocery Store,Bakery,Bubble Tea Shop
